<a href="https://colab.research.google.com/github/wr0124/Learning_essential/blob/main/CorssAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#source

old: https://github.com/huggingface/diffusers_all/blob/main/src/diffusers/models/attention.py

new: https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/attention_processor.py#L41

https://github.com/huggingface/diffusers/blob/main/src/diffusers/models/attention.py

branch: https://github.com/huggingface/diffusers/blob/4125756e88e82370c197fecf28e9f0b4d7eee6c3/src/diffusers/models/cross_attention.py

other:
show attention: https://github.com/google/prompt-to-prompt/blob/main/prompt-to-prompt_ldm.ipynb



# Import

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import numpy as np

#1.self_attention

##1.1alg

In [77]:
class SelfAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())
        #print("W_query shape:", self.W_query)
        #print("W_key shape:", self.W_key)
        #print("W_value shape:", self.W_value)



    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        #print("keys ", keys)
        #print("queries ", queries)
        #print("values ", values)
        attn_scores = queries @ keys.transpose(-2,-1)  # unnormalized attention weights
        #print("attn_scores ", attn_scores )
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1
        )

        #print("attn_weights ", attn_weights)
        context_vec = attn_weights @ values
        #print("context_vec ", context_vec)
        return context_vec


class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [SelfAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        print("heads ", dir(self.heads[0]))

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [78]:
# Example usage for MultiHeadAttentionWrapper
d = 3
d_k_q = 2
d_v = 2
num_heads = 2
batch_size = 1
seq_len = 6

multi_head_attention = MultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
x = torch.randint(low=0,high=d,size=(batch_size, sequence_length, d)).float()
#print("MultiHeadAttentionWrapper input shape:", x)
output = multi_head_attention(x)
#print("MultiHeadAttentionWrapper output shape:", output)
print("MultiHeadAttentionWrapper output shape:" , output.shape )
#output shape [batch_size, seq_len, num_heads * d_v]


heads  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameter

In [21]:
A=torch.tensor([[1,2,3],[4,5,6]])
print(A)
print(A.shape)
B=torch.tensor([[4,5,6],[7,8,9]])
print(B)
print(B.shape)
print(torch.cat((A,B),dim=-1))

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])
tensor([[4, 5, 6],
        [7, 8, 9]])
torch.Size([2, 3])
tensor([[1, 2, 3, 4, 5, 6],
        [4, 5, 6, 7, 8, 9]])


##1.2two methods compare

In [23]:
# Define the selfttention class from Method 1
class SelfAttention(nn.Module):
    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())
        print("W_query shape:", self.W_query)
        print("W_key shape:", self.W_key)
        print("W_value shape:", self.W_value)

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        #print("keys ", keys)
        #print("queries ", queries)
        #print("values ", values)
        attn_scores = queries @ keys.transpose(-2,-1)  # unnormalized attention weights
        #print("attn_scores ", attn_scores )
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1
        )

        #print("attn_weights ", attn_weights)
        context_vec = attn_weights @ values
        #print("context_vec ", context_vec)
        return context_vec

class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [SelfAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        #print("heads ", self.heads)

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

##################
# Define the QKVAttention class from Method 2
class QKVAttention(nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        self.n_heads = n_heads

    def forward(self, qkv):
        bs, width, length = qkv.shape
        assert width % (3 * self.n_heads) == 0
        ch = width // (3 * self.n_heads)
        q, k, v = qkv.chunk(3, dim=1)
        scale = 1 / math.sqrt(math.sqrt(ch))
        weight = torch.einsum(
            "bct,bcs->bts",
            (q * scale).view(bs * self.n_heads, ch, length),
            (k * scale).view(bs * self.n_heads, ch, length),
        )  # More stable with f16 than dividing afterwards
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum(
            "bts,bcs->bct", weight, v.reshape(bs * self.n_heads, ch, length)
        )
        return a.reshape(bs, -1, length)

# Example parameters
d = 4
d_k_q = 2
d_v = 2
num_heads = 2
seq_len = 6
batch_size = 1

# Example input
x = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("MultiHeadAttentionWrapper input shape:", x)

# Initialize Method 1
multihead_attention_1 = MultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
output_1 = multihead_attention_1(x)

# Initialize Method 2
# For Method 2, we need to prepare qkv input
# We'll use the same weights as in Method 1 to ensure identical outputs
queries = [x @ head.W_query for head in multihead_attention_1.heads]
keys = [x @ head.W_key for head in multihead_attention_1.heads]
values = [x @ head.W_value for head in multihead_attention_1.heads]
print("queries before cat ", queries)

# Concatenate Q, K, V for Method 2 input
queries = torch.cat(queries, dim=-1)
keys = torch.cat(keys, dim=-1)
values = torch.cat(values, dim=-1)
print("queries after cat", queries)

# Stack the heads to prepare qkv input
print("before permute qkv", torch.cat((queries, keys, values), dim=-1) )
qkv = torch.cat((queries, keys, values), dim=-1).permute(0, 2, 1)
print("after permute qkv", qkv  )


# Initialize and run QKVAttention
qkv_attention = QKVAttention(num_heads)
output_2 = qkv_attention(qkv)
#output shape [batch_size, num_heads * d_v,seq_len], here d_out_v = d_k_q

# Permute output_1 to match output_2 shape
output_1 = output_1.permute(0, 2, 1)

# Compare the outputs
print("Output from Method 1:\n", output_1, output_1.shape)
print("Output from Method 2:\n", output_2, output_2.shape)

# Check if the outputs are close enough
print("Are the outputs close? ", torch.allclose(output_1, output_2, atol=1e-6))


MultiHeadAttentionWrapper input shape: tensor([[[2., 0., 2., 0.],
         [0., 0., 1., 1.],
         [0., 1., 0., 1.],
         [3., 2., 3., 2.],
         [3., 2., 2., 3.],
         [0., 2., 3., 3.]]])
W_query shape: Parameter containing:
tensor([[3., 1.],
        [1., 0.],
        [1., 2.],
        [3., 3.]], requires_grad=True)
W_key shape: Parameter containing:
tensor([[2., 2.],
        [1., 3.],
        [3., 0.],
        [0., 3.]], requires_grad=True)
W_value shape: Parameter containing:
tensor([[0., 1.],
        [3., 3.],
        [3., 3.],
        [3., 2.]], requires_grad=True)
W_query shape: Parameter containing:
tensor([[1., 1.],
        [2., 1.],
        [1., 2.],
        [2., 0.]], requires_grad=True)
W_key shape: Parameter containing:
tensor([[1., 3.],
        [0., 1.],
        [0., 3.],
        [0., 1.]], requires_grad=True)
W_value shape: Parameter containing:
tensor([[2., 1.],
        [2., 1.],
        [2., 3.],
        [3., 2.]], requires_grad=True)
queries before cat  [

##1.3attention_map
https://github.com/google/prompt-to-prompt/blob/main/prompt-to-prompt_stable.ipynb

#2.CrossAttention

##2.1alg

In [27]:
class CrossAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

    def forward(self, x_1, x_2):           # x_2 is new
        queries_1 = x_1 @ self.W_query

        keys_2 = x_2 @ self.W_key          # new
        values_2 = x_2 @ self.W_value      # new

        attn_scores = queries_1 @ keys_2.transpose(-2, -1) # new
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1)

        context_vec = attn_weights @ values_2
        return context_vec

class CrossMultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [CrossAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        print("heads ", dir(self.heads[0]))

    def forward(self, x_1, x_2):
        return torch.cat([head(x_1, x_2) for head in self.heads], dim=-1)



In [28]:
# Example usage for CrossMultiHeadAttentionWrapper
d = 3
d_k_q = 2
d_v = 2
num_heads = 2
batch_size = 1
seq_len = 6

cross_multi_head_attention = CrossMultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
x_1 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
x_2 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("MultiHeadAttentionWrapper input1 shape:", x_1)
print("MultiHeadAttentionWrapper input2 shape:", x_2)
output = cross_multi_head_attention(x_1,x_2)
print("MultiHeadAttentionWrapper output shape:", output)
print("MultiHeadAttentionWrapper output shape:" , output.shape )
#output shape [batch_size, seq_len, num_heads * d_v]



heads  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameter

##2.2 two methods compare

In [29]:
# Define the QKVAttention class from Method 1
class CrossAttention(nn.Module):

    def __init__(self, d, d_k_q, d_v):
        super().__init__()
        self.d_out_kq = d_k_q
        self.W_query = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_key   = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_k_q)).float())
        self.W_value = nn.Parameter(torch.randint(low=0, high=d, size=(d, d_v)).float())

    def forward(self, x_1, x_2):           # x_2 is new
        queries_1 = x_1 @ self.W_query

        keys_2 = x_2 @ self.W_key          # new
        values_2 = x_2 @ self.W_value      # new

        attn_scores = queries_1 @ keys_2.transpose(-2, -1) # new
        attn_weights = torch.softmax(
            attn_scores / self.d_out_kq**0.5, dim=-1)

        context_vec = attn_weights @ values_2
        return context_vec

class CrossMultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d, d_k_q, d_v, num_heads):
        super().__init__()
        self.heads = nn.ModuleList(
            [CrossAttention(d, d_k_q, d_v)
             for _ in range(num_heads)]
        )
        print("heads ", dir(self.heads[0]))

    def forward(self, x_1, x_2):
        return torch.cat([head(x_1, x_2) for head in self.heads], dim=-1)



##################
# Define the QKVAttention class from Method 2
class CrossQKVAttention(nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        self.n_heads = n_heads

    def forward(self, qkv):
        bs, width, length = qkv.shape
        assert width % (3 * self.n_heads) == 0
        ch = width // (3 * self.n_heads)
        q, k, v = qkv.chunk(3, dim=1)
        scale = 1 / math.sqrt(math.sqrt(ch))
        weight = torch.einsum(
            "bct,bcs->bts",
            (q * scale).view(bs * self.n_heads, ch, length),
            (k * scale).view(bs * self.n_heads, ch, length),
        )  # More stable with f16 than dividing afterwards
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum(
            "bts,bcs->bct", weight, v.reshape(bs * self.n_heads, ch, length)
        )
        return a.reshape(bs, -1, length)

# Example parameters
# Example parameters
d = 4
d_k_q = 2
d_v = 2
num_heads = 2
seq_len = 6
batch_size = 1

# Example input
x_1 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
x_2 = torch.randint(low=0,high=d,size=(batch_size, seq_len, d)).float()
print("CrossMultiHeadAttentionWrapper input 1 shape:", x_1)
print("CrossMultiHeadAttentionWrapper input 2 shape:", x_2)

# Initialize Method 1
cross_multi_head_attention = CrossMultiHeadAttentionWrapper(d, d_k_q, d_v, num_heads)
output = cross_multi_head_attention(x_1,x_2)
print("MultiHeadAttentionWrapper output shape:", output)
print("MultiHeadAttentionWrapper output shape:" , output.shape )

print("-----------------------------------------------------")
# Initialize Method 2
# For Method 2, we need to prepare qkv input
# We'll use the same weights as in Method 1 to ensure identical outputs
queries = [x_1 @ head.W_query for head in cross_multi_head_attention.heads]
keys = [x_2 @ head.W_key for head in cross_multi_head_attention.heads]
values = [x_2 @ head.W_value for head in cross_multi_head_attention.heads]
print("queries before cat ", queries)

# Concatenate Q, K, V for Method 2 input
queries = torch.cat(queries, dim=-1)
keys = torch.cat(keys, dim=-1)
values = torch.cat(values, dim=-1)
print("queries after cat", queries)

# Stack the heads to prepare qkv input
print("before permute qkv", torch.cat((queries, keys, values), dim=-1) )
qkv = torch.cat((queries, keys, values), dim=-1).permute(0, 2, 1)
print("after permute qkv", qkv  )

# Initialize and run QKVAttention
qkv_attention = QKVAttention(num_heads)
output_2 = qkv_attention(qkv)
#output shape [batch_size, num_heads * d_v,seq_len], here d_out_v = d_k_q

# Permute output_1 to match output_2 shape
output_1 = output_1.permute(0, 2, 1)

# Compare the outputs
print("Output from Method 1:\n", output_1)
print("Output from Method 2:\n", output_2)

# Check if the outputs are close enough
#print("Are the outputs close? ", torch.allclose(output_1, output_2, atol=1e-6))


CrossMultiHeadAttentionWrapper input 1 shape: tensor([[[2., 0., 3., 2.],
         [3., 1., 0., 1.],
         [3., 3., 0., 3.],
         [2., 1., 2., 3.],
         [2., 3., 1., 1.],
         [3., 3., 3., 3.]]])
CrossMultiHeadAttentionWrapper input 2 shape: tensor([[[0., 2., 0., 0.],
         [0., 2., 2., 1.],
         [3., 3., 2., 2.],
         [2., 3., 3., 2.],
         [3., 0., 3., 2.],
         [0., 0., 0., 3.]]])
heads  ['T_destination', 'W_key', 'W_query', 'W_value', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '

RuntimeError: The size of tensor a (4) must match the size of tensor b (6) at non-singleton dimension 2